In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

## create dataset

In [2]:
PATH='data/aclImdb/'
names = ['neg','pos']
names1 = ['neg', 'pos_']
names2 = ['neg_', 'pos']

In [3]:
! ls {PATH}train

labeledBow.feat pos             unsupBow.feat   urls_pos.txt
neg             unsup           urls_neg.txt    urls_unsup.txt


In [4]:
trn1,trn1_y = texts_labels_from_folders(f'{PATH}train',names1)
val1,val1_y = texts_labels_from_folders(f'{PATH}test',names1)

In [5]:
trn2,trn2_y = texts_labels_from_folders(f'{PATH}train',names2)
val2,val2_y = texts_labels_from_folders(f'{PATH}test',names2)

In [6]:
len(trn1), len(trn1_y), len(trn2), len(trn2_y)

(12500, 12500, 12500, 12500)

In [7]:
assert (trn1_y==0).all()
(trn1_y==0).all()

True

In [8]:
assert (trn1_y==0).all()
(val1_y==0).all()

True

In [9]:
assert (trn2_y==1).all()
(trn2_y==1).all()

True

In [10]:
assert (val2_y==1).all()
(val2_y==1).all()

True

In [11]:
def sample_util(data, label, n):
    assert len(data) == label.shape[0]
    idx = np.random.choice(range(len(data)),n)
    data_new = [data[i] for i in idx]
    label_new = label[idx]
    return data_new, label_new

In [12]:
def sample(data1, label1, data2, label2, n):
    t1, t2 = sample_util(data1, label1, n)
    t3, t4 = sample_util(data2, label2, n)
    data = t1 + t3
    label = np.concatenate((t2, t4))
    return data, label

In [13]:
trn, trn_y = sample(trn1, trn1_y, trn2, trn2_y, 64*10)
val, val_y = sample(val1, val1_y, val2, val2_y, 64*20)

In [14]:
# trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
# val,val_y = texts_labels_from_folders(f'{PATH}test',names)

## create vectors and vocab

In [15]:
veczr = CountVectorizer(tokenizer=tokenize)

In [16]:
trn_term_doc = veczr.fit_transform(trn)

In [17]:
val_term_doc = veczr.transform(val)

In [18]:
vocab = veczr.get_feature_names(); vocab[5000:5005]

['dictatorship', 'diction', 'dictionary', 'did', 'diddle']

In [19]:
vocab[:5]

['!', '"', '#', '$', '%']

In [20]:
len(vocab)

20001

In [21]:
trn_term_doc = trn_term_doc.sign()

In [22]:
val_term_doc = val_term_doc.sign()

## Naive Bayes

We define the **log-count ratio** $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive document has a feature divided by the number of positive documents.

In [23]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return p+1

In [24]:
x=trn_term_doc
y=trn_y

p = pr(1)/pr(1).sum()
q = pr(0)/pr(0).sum()
r = np.log(p/q)
b = np.log((y==1).mean() / (y==0).mean())

In [25]:
val_term_doc.shape

(2560, 20001)

In [26]:
val_term_doc.shape[1]

20001

In [27]:
x.shape

(1280, 20001)

In [28]:
r.shape

(1, 20001)

In [29]:
b

0.0

In [30]:
val_term_doc

<2560x20001 sparse matrix of type '<class 'numpy.int64'>'
	with 352524 stored elements in Compressed Sparse Row format>

In [31]:
(val_term_doc @ r.T).shape

(2560, 1)

In [32]:
val_term_doc @ r.T

matrix([[-10.22232],
        [ -3.28149],
        [ -7.90024],
        ...,
        [  8.47319],
        [  8.31353],
        [ 10.08913]])

In [33]:
val_y

array([0, 0, 0, ..., 1, 1, 1])

In [34]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.80625

In [35]:
pre_preds = val_term_doc @ np.stack([np.log(p), np.log(q)]).T + b

In [36]:
pre_preds

matrix([[ -897.68999,  -887.46767],
        [-1821.03818, -1817.75669],
        [ -308.38326,  -300.48302],
        ...,
        [ -952.47611,  -960.9493 ],
        [-1016.18753, -1024.50106],
        [ -705.86977,  -715.95891]])

In [37]:
preds = pre_preds.T[0] > pre_preds.T[1]
(preds==val_y).mean()

0.80625

In [38]:
type(val_term_doc)

scipy.sparse.csr.csr_matrix

In [39]:
val_term_doc[0]

<1x20001 sparse matrix of type '<class 'numpy.int64'>'
	with 115 stored elements in Compressed Sparse Row format>

In [40]:
xx = val_term_doc[0]

In [41]:
xx.toarray().shape

(1, 20001)

In [42]:
[vocab[index] for index, i in enumerate(xx.toarray()[0]) if i > 0]

['"',
 "'",
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '>',
 'a',
 'adult',
 'agent',
 'ahead',
 'an',
 'and',
 'anderson',
 'any',
 'as',
 'at',
 'awful',
 'bad',
 'begin',
 'being',
 'best',
 'between',
 'block',
 'boring',
 'br',
 'brings',
 'but',
 'called',
 'cartoon',
 'cartoons',
 'city',
 'clever',
 'complete',
 'concept',
 'decent',
 'disappointment',
 'down',
 'duck',
 'episodes',
 'even',
 'failure',
 'first',
 'for',
 'from',
 'funny',
 'gags',
 'gary',
 'good',
 'goofy',
 'gross',
 'her',
 'homage',
 'if',
 'is',
 'its',
 'just',
 'lacked',
 'like',
 'make',
 'mediocre',
 'men',
 'mostly',
 'neither',
 'network',
 'new',
 'of',
 'one',
 'or',
 'other',
 'out',
 'pamela',
 'period',
 'plain',
 'plot',
 'pretty',
 'rat',
 'real',
 'rebirth',
 's',
 'saves',
 'see',
 'seems',
 'series',
 'sexy',
 'she',
 'shock',
 'so',
 'some',
 'spoilers',
 'spoof',
 'spoofs',
 'suffered',
 'superhero',
 'superior',
 'taped',
 'the',
 'this',
 'those',
 'tick',
 'to',
 'tries',
 'unwatchabl

In [43]:
total = 0
for index, i in enumerate(xx.toarray()[0]):
    if i:
        total += 1
total

115

## Logistic regression (sklearn)

Here is how we can fit logistic regression where the features are the unigrams.

In [44]:
LogisticRegression

sklearn.linear_model._logistic.LogisticRegression

In [45]:
m = LogisticRegression(C=1e8, dual=False, max_iter=1000)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.828515625

...and the regularized version

In [46]:
m = LogisticRegression(C=1.0, dual=False, max_iter=1000)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.831640625

## Logistic regression (PyTorch)

In [47]:
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.nlp import *

import torch.nn as nn

In [48]:
class MySimpleNB(nn.Module):
    def __init__(self, nf, ny):
        super().__init__()
        self.w = nn.Embedding(nf, ny)
        # self.w = nn.Embedding(nf+1, ny)
        # self.w.weight.data.uniform_(-1, 1)
        self.w.weight.data = torch.FloatTensor(r.tolist()[0])
        self.w.weight.data = self.w.weight.data.reshape(-1, 1)
        # self.r = nn.Embedding(nf, ny)
        
    def forward(self, feat_idx):
        # self.w.weight.data[0] = 0
        idx = feat_idx - 1
        idx2 = [a for a in idx if a >= 0]
        idx3 = np.array(idx2)
        v = self.w(V(idx3))
        # r = self.r(feat_idx)
        # x = ((w+self.w_adj)*r/self.r_adj).sum(1)
        # x = w*r
        x = v.sum(1)
        # return F.softmax(x)
        # return x.reshape(1, -1)
        return x

In [49]:
def binary_loss(pred, y):
    # y2 = torch.max(y,axis=1)[0]
    y2 = np.argmax(y)
    p = torch.exp(pred) / (1+torch.exp(pred))
    result = torch.mean(-(y2 * torch.log(p) + (1-y2)*torch.log(1-p)))
    # return result.reshape(1, -1)
    return result

In [50]:
r.shape

(1, 20001)

In [51]:
len(vocab)

20001

In [52]:
net3 = MySimpleNB(len(vocab), 1)

In [53]:
net3.w.weight.data.shape

torch.Size([20001, 1])

In [54]:
r.shape

(1, 20001)

In [55]:
sl=val_term_doc.shape[1]
sl

20001

In [56]:
sl=val_term_doc.shape[1]
md = TextClassifierData.from_bow(
    trn_term_doc, trn_y,
    val_term_doc, val_y,
    200,
)

In [57]:
# ??TextClassifierData.from_bow

In [58]:
trn_term_doc.shape

(1280, 20001)

In [59]:
ii=0

In [60]:
net4 = MySimpleNB(len(vocab), 1)
# loss = nn.NLLLoss()
# loss = torch.nn.CrossEntropyLoss()
loss = binary_loss
lr = 1e-2
losses=[]

In [61]:
net4.w.weight

Parameter containing:
tensor([[-9.0411e-04],
        [-6.0760e-02],
        [ 1.6750e-01],
        ...,
        [ 7.0650e-01],
        [ 1.1120e+00],
        [ 7.0650e-01]], requires_grad=True)

In [62]:
ii=2

In [63]:
t = md.trn_ds[ii]
ii = ii + 1
xt, _a, _b, yt = t

In [64]:
t

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     2,
            7,    12,    13,    14,    15,   117,   290,   292,   298,   337,   449,   450,   467,   633,
          895,   947,  1218,  1302,  1748,  1889,  1906,  1911,  2140,  2202,  2293,  2296,  2627,  2644,
         2694,  2716,  2718,  2984,  3031,  3053,  3358,  3506,  3578,  3630,  4079,  4128,  4230,  4267,
         4289,  4389,  4606,  4864,  4881,  5004,  5070,  5077,  5351,  5371,  5833,  6444,  6633,  6651,
         6761,  6786,  6961,  7011,  7217,  7218,  7618,  7635,  8112,  8158,  8188,  8326,  8628,  8635,
         8801,  8952,  9451,  9474,  9572,  98

In [65]:
xt.shape, len(vocab)

((200,), 20001)

In [66]:
xt.shape

(200,)

In [67]:
vocab[18178], len(vocab)

('trades', 20001)

In [68]:
_a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [69]:
_a.sum()

145

In [70]:
_b

145

In [71]:
yt

array([1., 0.], dtype=float32)

In [72]:
md.trn_ds[ii]

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            1,     7,     8,     9,    12,    14,    15,    23,    24,    66,   152,   153,   154,   194,
          265,   290,   292,   298,   322,   540,   693,   737,   895,  1117,  1168,  1218,  1237,  1302,
         1713,  1764,  1803,  1817,  1943,  1953,  2296,  2451,  2600,  2873,  2990,  3177,  3583,  4094,
         4158,  4460,  4873,  5006,  5653,  6004,  6063,  6083,  6237,  6391,  6460,  6470,  6546,  6553,
         6633,  6649,  6662,  6761,  6829,  6876,  6957,  7011,  7115,  7413,  7419,  7639,  7974,  8103,
         8186,  8208,  8433,  8505,  8752,  8801,  8952,  9451,  9459,  9474,  9483,  9572,  9675,  9749,
         9833,  9979, 10071, 10171, 10236, 103

In [73]:
xt.shape

(200,)

In [74]:
for index, idx in enumerate(to_np(xt)):
    if idx:
        print(vocab[idx-1])

"
'
,
-
.
/
1970s
<
>
a
about
actors
actress
adapted
ago
and
ann
as
at
because
best
better
between
body
book
boyle
br
by
cable
camera
can
canadian
center
chance
character
clearly
col
combined
commercials
cosmetic
course
creepiness
critical
cross
curve
decided
deserved
desperate
did
directed
director
door
dose
else
eyebrows
features
feeling
film
find
flynn
for
from
front
goes
gone
hardly
has
having
her
house
houswives
if
in
is
it
jeff
kennedy
know
known
lara
lifetime
like
long
look
made
mainly
material
mention
mild
moments
movie
ms
mystery
new
next
no
not
novel
of
on
one
or
other
parts
precede
read
recognizable
rest
rivers
s
same
serves
setting
showed
siddons
someone
sports
star
stepford
t
take
television
that
the
thing
this
threw
through
time
title
to
transformation
version
viewer
was
watched
we
what
when
which
who
with
wives
woolnough
you


In [75]:
vocab[0], len(vocab)

('!', 20001)

In [76]:
xt

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     2,
           7,    12,    13,    14,    15,   117,   290,   292,   298,   337,   449,   450,   467,   633,
         895,   947,  1218,  1302,  1748,  1889,  1906,  1911,  2140,  2202,  2293,  2296,  2627,  2644,
        2694,  2716,  2718,  2984,  3031,  3053,  3358,  3506,  3578,  3630,  4079,  4128,  4230,  4267,
        4289,  4389,  4606,  4864,  4881,  5004,  5070,  5077,  5351,  5371,  5833,  6444,  6633,  6651,
        6761,  6786,  6961,  7011,  7217,  7218,  7618,  7635,  8112,  8158,  8188,  8326,  8628,  8635,
        8801,  8952,  9451,  9474,  9572,  9849,  9979,

In [77]:
net4.w(V(xt)).sum()

tensor(15.0486, grad_fn=<SumBackward0>)

In [78]:
xt.shape

(200,)

In [79]:
xt.shape

(200,)

In [80]:
y_pred = net4(V(xt))
print(y_pred)
l = binary_loss(y_pred, yt)

tensor([-0.0608, -0.0324,  0.0052, -0.0278,  0.0086, -0.0434, -1.0853, -0.0374,
        -0.0321, -0.0027, -0.0833,  0.0133,  0.1957,  0.4188, -0.1834,  0.0295,
         0.0133,  0.0537, -0.0744, -0.0341,  0.7328, -0.3003,  0.1957, -0.2098,
        -0.0473,  0.0133, -0.0322,  0.0274, -1.4907, -0.4975,  0.0210,  1.1120,
         0.3700,  0.1777,  0.1321, -0.3496,  0.4188,  0.0133,  0.9296, -0.6798,
         0.4188, -0.6798,  0.4188,  0.1957, -0.6798, -0.7243,  0.0133,  0.7065,
        -0.1837,  0.0461, -0.2878, -1.0365, -0.4975, -0.4627, -1.3729,  0.5242,
        -0.3775, -0.0317,  0.0934,  0.5242, -0.0437,  0.0400,  0.1564, -0.2335,
        -0.0776, -0.4975,  0.0490, -0.0722, -0.0344, -0.1739, -0.6798, -0.1993,
         0.0222,  0.0391,  0.0116, -0.5744,  0.2365, -0.1645,  0.6459,  0.0133,
         0.1957, -0.1637, -0.1060, -0.3544, -0.3549, -0.4285, -0.3661, -0.4662,
        -1.2394,  0.1134, -0.1883, -0.3231,  0.5634,  0.3971, -0.4022, -0.4183,
        -0.1570,  0.2140, -0.0063, -0.08

In [81]:
y_pred

tensor([-0.0608, -0.0324,  0.0052, -0.0278,  0.0086, -0.0434, -1.0853, -0.0374,
        -0.0321, -0.0027, -0.0833,  0.0133,  0.1957,  0.4188, -0.1834,  0.0295,
         0.0133,  0.0537, -0.0744, -0.0341,  0.7328, -0.3003,  0.1957, -0.2098,
        -0.0473,  0.0133, -0.0322,  0.0274, -1.4907, -0.4975,  0.0210,  1.1120,
         0.3700,  0.1777,  0.1321, -0.3496,  0.4188,  0.0133,  0.9296, -0.6798,
         0.4188, -0.6798,  0.4188,  0.1957, -0.6798, -0.7243,  0.0133,  0.7065,
        -0.1837,  0.0461, -0.2878, -1.0365, -0.4975, -0.4627, -1.3729,  0.5242,
        -0.3775, -0.0317,  0.0934,  0.5242, -0.0437,  0.0400,  0.1564, -0.2335,
        -0.0776, -0.4975,  0.0490, -0.0722, -0.0344, -0.1739, -0.6798, -0.1993,
         0.0222,  0.0391,  0.0116, -0.5744,  0.2365, -0.1645,  0.6459,  0.0133,
         0.1957, -0.1637, -0.1060, -0.3544, -0.3549, -0.4285, -0.3661, -0.4662,
        -1.2394,  0.1134, -0.1883, -0.3231,  0.5634,  0.3971, -0.4022, -0.4183,
        -0.1570,  0.2140, -0.0063, -0.08

In [82]:
print(net4.w.weight.grad)

None


In [83]:
l.shape

torch.Size([])

In [84]:
l.backward()

In [85]:
_a.shape, _a

((200,),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [86]:
for idx, i in enumerate(net4.w.weight.grad.data):
    for j in i:
        if j != 0:
            print(idx, j, net4.w.weight.data[idx])

1 tensor(0.0033) tensor([-0.0608])
6 tensor(0.0034) tensor([-0.0324])
11 tensor(0.0035) tensor([0.0052])
12 tensor(0.0034) tensor([-0.0278])
13 tensor(0.0035) tensor([0.0086])
14 tensor(0.0034) tensor([-0.0434])
116 tensor(0.0017) tensor([-1.0853])
289 tensor(0.0034) tensor([-0.0374])
291 tensor(0.0034) tensor([-0.0321])
297 tensor(0.0034) tensor([-0.0027])
336 tensor(0.0033) tensor([-0.0833])
448 tensor(0.0035) tensor([0.0133])
449 tensor(0.0038) tensor([0.1957])
466 tensor(0.0042) tensor([0.4188])
632 tensor(0.0031) tensor([-0.1834])
894 tensor(0.0035) tensor([0.0295])
946 tensor(0.0035) tensor([0.0133])
1217 tensor(0.0035) tensor([0.0537])
1301 tensor(0.0033) tensor([-0.0744])
1747 tensor(0.0034) tensor([-0.0341])
1888 tensor(0.0047) tensor([0.7328])
1905 tensor(0.0029) tensor([-0.3003])
1910 tensor(0.0038) tensor([0.1957])
2139 tensor(0.0031) tensor([-0.2098])
2201 tensor(0.0034) tensor([-0.0473])
2292 tensor(0.0035) tensor([0.0133])
2295 tensor(0.0034) tensor([-0.0322])
2626 tenso

In [87]:
net4.w.weight.data -= net4.w.weight.grad.data * lr

In [88]:
lr

0.01

In [89]:
net4.w.weight.data

tensor([[-9.0411e-04],
        [-6.0793e-02],
        [ 1.6750e-01],
        ...,
        [ 7.0650e-01],
        [ 1.1120e+00],
        [ 7.0650e-01]])

In [90]:
net4.w.weight.grad.data

tensor([[0.0000],
        [0.0033],
        [0.0000],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]])

In [91]:
for i in net4.w.weight.grad.data:
    for j in i:
        if j != 0:
            print(j)

tensor(0.0033)
tensor(0.0034)
tensor(0.0035)
tensor(0.0034)
tensor(0.0035)
tensor(0.0034)
tensor(0.0017)
tensor(0.0034)
tensor(0.0034)
tensor(0.0034)
tensor(0.0033)
tensor(0.0035)
tensor(0.0038)
tensor(0.0042)
tensor(0.0031)
tensor(0.0035)
tensor(0.0035)
tensor(0.0035)
tensor(0.0033)
tensor(0.0034)
tensor(0.0047)
tensor(0.0029)
tensor(0.0038)
tensor(0.0031)
tensor(0.0034)
tensor(0.0035)
tensor(0.0034)
tensor(0.0035)
tensor(0.0013)
tensor(0.0026)
tensor(0.0035)
tensor(0.0052)
tensor(0.0041)
tensor(0.0038)
tensor(0.0037)
tensor(0.0029)
tensor(0.0042)
tensor(0.0035)
tensor(0.0049)
tensor(0.0023)
tensor(0.0042)
tensor(0.0023)
tensor(0.0042)
tensor(0.0038)
tensor(0.0023)
tensor(0.0023)
tensor(0.0035)
tensor(0.0046)
tensor(0.0031)
tensor(0.0035)
tensor(0.0030)
tensor(0.0018)
tensor(0.0026)
tensor(0.0027)
tensor(0.0014)
tensor(0.0043)
tensor(0.0028)
tensor(0.0034)
tensor(0.0036)
tensor(0.0043)
tensor(0.0034)
tensor(0.0035)
tensor(0.0037)
tensor(0.0030)
tensor(0.0033)
tensor(0.0026)
tensor(0.0

In [92]:
net4.w.weight.grad.data.zero_()

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [93]:
from datetime import datetime

In [94]:
len(md.trn_ds)

1280

In [95]:
a1, a2, a3, a4 = md.trn_ds[0]

In [96]:
a1

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     2,     7,    12,    14,   298,   448,
         455,   737,   774,   787,   809,   878,   895,  1218,  1302,  1748,  1764,  1934,  2304,  2543,
        3037,  3244,  3255,  3440,  3845,  4538,  5006,

In [97]:
a2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [98]:
a3

94

In [99]:
a4, np.argmax(a4)

(array([1., 0.], dtype=float32), 0)

In [100]:
torch.max(V(a4))

tensor(1.)

In [101]:
loss(net4(V(a1)), V(a4))

tensor(0.6429, grad_fn=<MeanBackward0>)

In [102]:
net4(V(a1))

tensor([-0.0608, -0.0325,  0.0052,  0.0086, -0.0027,  0.3953, -0.2895, -0.0344,
        -0.0332,  0.3114,  0.0133,  0.0294,  0.0294,  0.0536, -0.0745, -0.0342,
        -0.2027, -1.0853, -0.9029, -1.3158,  0.0133, -0.1690, -0.4975, -1.2394,
        -1.0853,  0.1267, -0.3107, -0.4975, -0.7716, -0.2549,  0.6345, -0.0354,
        -1.0853, -0.1014, -0.0437, -0.1492, -0.3921,  0.1629, -0.0722,  0.2575,
         0.2247, -0.3231, -0.2125,  0.1311, -0.0892,  0.7065,  0.0934,  0.0222,
         0.0391,  0.0116, -0.1767,  0.2422,  0.0913, -0.3550, -0.1699, -0.1884,
         0.0565, -0.2685, -0.9481, -0.0162, -0.0063, -0.0875, -0.0399, -0.2002,
        -0.1873,  0.2365,  0.2365,  0.2044,  0.6683,  0.0155, -0.3284,  0.1462,
         0.0133, -0.1759, -0.1408,  0.0133, -0.0763, -0.2093, -0.0457,  0.0102,
        -0.2287,  0.1257, -0.0589, -0.0846,  0.0393, -0.0199, -0.2790, -1.7784,
        -0.1157, -0.0311, -0.1275,  0.0403, -0.1808, -1.0853],
       grad_fn=<SumBackward1>)

In [103]:
def score2(x, y):
    y_pred = to_np(net2(V(x)))
    return np.sum(y_pred.argmax(axis=1) == to_np(y).argmax(axis=1))/len(y_pred)

In [104]:
def score(x, y):
    # print(f'x={x}, y={y}')
    y_pred = to_np(net2(V(x))).sum() >= 0
    # print(f'y_pred={y_pred}')
    y2 = np.argmax(y)
    # print(f'y2={y2}')
    return np.sum(y_pred == y2)

In [105]:
from tqdm import notebook

In [106]:
datetime.now()

datetime.datetime(2020, 1, 5, 19, 55, 58, 865927)

In [107]:
net2 = MySimpleNB(len(vocab), 1)
# loss = nn.NLLLoss()
# loss = torch.nn.CrossEntropyLoss()
loss = binary_loss
# lr = 1e-0
lr = 1e-3
train_loss_list = []
val_loss_list = []
val_acc_list= []
loss_list = []

In [108]:
(net2.w(V(trn_term_doc.toarray()))).shape

torch.Size([1280, 20001, 1])

In [109]:
(net2.w(V(trn_term_doc.toarray()))).sum(1).shape

torch.Size([1280, 1])

In [110]:
(net2.w(V(trn_term_doc.toarray()))).sum(1)

tensor([[-23.7094],
        [-25.0861],
        [-26.7620],
        ...,
        [-23.8291],
        [-24.9664],
        [-28.1986]], grad_fn=<SumBackward1>)

In [111]:
(trn_term_doc @ r.T > 0).shape

(1280, 1)

In [112]:
trn_term_doc @ r.T

matrix([[-13.59826],
        [-16.91834],
        [-15.28112],
        ...,
        [ -2.12701],
        [  1.68547],
        [  9.05191]])

In [113]:
val_scores = []
for t in tqdm(md.val_ds, total=len(trn_term_doc)):
    x, _a, _b, y = t
    val_scores.append(score(x,y))
np.mean(to_np(val_scores))

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [ ]:
val_scores = []
for t in tqdm(md.val_ds, total=len(md.val_ds)):
    x, _a, _b, y = t
    val_scores.append(score(x,y))
np.mean(to_np(val_scores))

In [ ]:
import os
filename = 'acc.txt'
try:
    os.remove(filename)
except:
    pass

In [ ]:
len(md.trn_dl)

In [ ]:
print(f'lr={lr}')
f = open(filename, 'a')

for epoch in range(5):
    print('')
    print('epoch:', epoch)
    print('time:', datetime.now())
    loss_list = [0]
    for index, t in tqdm(enumerate(md.trn_ds), total=len(md.trn_ds)):
        xt, _a, _b, yt = t
        y_pred = net2(V(xt))
        l = loss(y_pred, V(yt))
        # l = loss(yt, y_pred)
        loss_list.append(l)
        # print(f'{index}, {l}, {datetime.now().time()}')

        # Backward pass: 
        # compute gradient of the loss with respect to 
        # model parameters
        l.backward()
        net2.w.weight.data -= net2.w.weight.grad.data * lr
        # net2.b.data -= net2.b.grad.data * lr
        
        net2.w.weight.grad.data.zero_()
        # net2.b.grad.data.zero_()   

    if epoch % 1 == 0:
        train_scores = []
        for t in tqdm(md.trn_ds, total=len(md.trn_ds)):
            x, _a, _b, y = t
            train_scores.append(loss(net2(V(x)), V(y)))
        l2 = np.mean(to_np(train_scores))
        train_loss_list.append(l2)

        val_scores = []
        for t in tqdm(md.val_ds, total=len(md.val_ds)):
            x, _a, _b, y = t
            train_scores.append(loss(net2(V(x)), V(y)))
        l3 = np.mean(to_np(train_scores))
        val_loss_list.append(l3)
                
        val_scores = []
        for t in tqdm(md.val_ds, total=len(md.val_ds)):
            x, _a, _b, y = t
            val_scores.append(score(x,y))
        l4 = np.mean(to_np(val_scores))
        val_acc_list.append(l4)

        # print(f'epoch={epoch}, score={np.mean(val_scores)}')
        print(f'epoch={epoch}, score={l2}')
        print(f'epoch={epoch}, score={l3}')
        print(f'epoch={epoch}, score={l4}')
        f.write(f"{l2}\t{l3}\t{l4}\n")
        f.flush()
f.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
length=len(train_loss_list)
df = pd.DataFrame({
    'train':train_loss_list[:length], 
    'valid':val_loss_list[:length],
    'valid_acc':val_acc_list[:length]
})

In [ ]:
df.plot(subplots=True)

In [ ]:
plt.plot(loss_list)

In [ ]:
plt.plot(val_acc_list)

## Deep NB